# IMPORT LIBRARIES

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score

from sklearn.model_selection import GroupKFold
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, make_scorer

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# SET AND RESET ROWS AND COLS

In [38]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [82]:
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

# READ CSV FILE

In [100]:
df_probe = pd.read_csv('Probe_data.csv')
df_question = pd.read_csv('Question_data.csv')
df_overall = pd.read_csv('Overall_data_m.csv')

In [104]:
df_overall

,Participant,TUTMean,DisengageMean,AwarenessMean,FMTMean,TextRTMean,TestMean,ReadPrior,OffTaskStuck,OffTaskJump,...,Setting,Race,D1,D2,D3,D4,D5,D6,D7,D8
0,593890eac6aa16000101f037,0.428571,2.000000,5.428571,5.428571,8.150127,0.7,n,4,5,...,1,10,1,0,0,0,1,1,0,0
1,596e1af7a09655000197d4bb,0.000000,3.000000,6.714286,5.000000,8.258325,0.8,n,3,4,...,1,10,0,0,0,0,0,1,0,0
2,5af835d8e19f8c00019e6dc0,0.000000,1.000000,7.000000,6.857143,15.013760,0.8,n,4,3,...,1,10,0,0,0,0,0,1,0,0
3,5b07b71c68eff50001d1c859,0.285714,3.000000,5.000000,4.857143,8.980250,0.6,n,4,4,...,1,7,0,0,0,0,0,0,1,0
4,5bd7971b0aac450001f951aa,0.571429,1.000000,7.000000,7.000000,15.064745,0.8,n,3,3,...,1,10,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,6295270587394aac7ce600db,0.142857,4.000000,3.857143,4.428571,10.930252,0.7,n,4,5,...,1,10,1,0,0,0,1,1,0,0
350,63038702fea51556c37c36cf,0.571429,4.285714,5.142857,5.000000,7.708367,0.7,n,4,5,...,2,10,0,1,0,0,1,0,0,0
351,63d17cf704d2d7053d56b962,0.857143,2.857143,3.571429,3.428571,8.862255,0.8,n,2,2,...,1,10,0,1,0,0,0,0,0,0
352,63d3fa78d12b38b131ef6b76,1.000000,5.285714,7.000000,5.142857,8.167373,0.9,n,4,5,...,1,10,1,1,1,0,1,1,0,0


# AVERAGE RESPONSE TO TUT

In [43]:
df_probe['tut_1'] = df_probe.groupby('Participant')['TUT'].transform(lambda x: (x == 1).sum())
df_probe['tut_total'] = df_probe.groupby('Participant')['TUT'].transform('count')
df_probe['avg_response_to_tut'] = df_probe['tut_1']/df_probe['tut_total']

df_probe_extract = df_probe[['Participant', 'avg_response_to_tut']].drop_duplicates().reset_index(drop=True)
df_probe_extract

,Participant,avg_response_to_tut
0,593890eac6aa16000101f037,0.428571
1,596e1af7a09655000197d4bb,0.000000
2,5af835d8e19f8c00019e6dc0,0.000000
3,5b07b71c68eff50001d1c859,0.285714
4,5bd7971b0aac450001f951aa,0.571429
5,5c6477fe4f0ea90001782441,0.000000
6,5cb568ebf0e31e0017d9bf32,0.285714
7,5d4459c166881d001c4d5451,0.571429
8,5d5497857aff650001a18c3c,0.285714
9,5d79375d794d900017671c07,0.142857


# AVERAGE CORRECTNESS SCORE/ AVERAGE QUESTION RESPONSE TIME

In [56]:
df_question['correct_count'] = df_question.groupby('Participant')['Correct'].transform(lambda x: (x == 1).sum())
df_question['total_correct'] = df_question.groupby('Participant')['Correct'].transform('count')
df_question['avg_correctness_score'] = df_question['correct_count']/df_question['total_correct']

df_question['rt_sum'] = df_question.groupby('Participant')['RT'].transform(sum)
df_question['total_rt'] = df_question.groupby('Participant')['RT'].transform('count')
df_question['avg_question_rt'] = df_question['rt_sum']/df_question['total_rt']

df_question_extract = df_question[['Participant', 'avg_correctness_score', 'avg_question_rt']].drop_duplicates().reset_index(drop=True)
df_question_extract

,Participant,avg_correctness_score,avg_question_rt
0,593890eac6aa16000101f037,0.7,9.13734
1,596e1af7a09655000197d4bb,0.8,7.94115
2,5af835d8e19f8c00019e6dc0,0.8,15.83652
3,5b07b71c68eff50001d1c859,0.6,7.32070
4,5bd7971b0aac450001f951aa,0.8,28.15962
5,5c6477fe4f0ea90001782441,0.7,10.35345
6,5cb568ebf0e31e0017d9bf32,0.5,12.18166
7,5d4459c166881d001c4d5451,0.8,7.30917
8,5d5497857aff650001a18c3c,0.7,9.00064
9,5d79375d794d900017671c07,0.8,7.67636


# AVERAGE READING PARAGRAPH TIME

In [63]:
df_overall_extract = df_overall[['Participant', 'TextRTMean']]
df_overall_extract

,Participant,TextRTMean
0,593890eac6aa16000101f037,8.150127
1,596e1af7a09655000197d4bb,8.258325
2,5af835d8e19f8c00019e6dc0,15.013760
3,5b07b71c68eff50001d1c859,8.980250
4,5bd7971b0aac450001f951aa,15.064745
5,5c6477fe4f0ea90001782441,10.256725
6,5cb568ebf0e31e0017d9bf32,11.507962
7,5d4459c166881d001c4d5451,6.475487
8,5d5497857aff650001a18c3c,13.268745
9,5d79375d794d900017671c07,11.655997


# COUNT NUMBER OF PARTICIPANTS BEFORE MERGING

In [80]:
# Count number of participants
dataframes = [
    ("df_probe_extract", df_probe_extract),
    ("df_question_extract", df_question_extract),
    ("df_overall_extract", df_overall_extract)
]
def calculate_statistics(df, label_column='TUT'):
    num_participants = df['Participant'].nunique()
    return num_participants

for i, (name, df) in enumerate(dataframes, start=1):
    num_participants = calculate_statistics(df)
    print(f"{name}:")
    print(f"  Number of participants = {num_participants}")


df_probe_extract:
  Number of participants = 354
df_question_extract:
  Number of participants = 354
df_overall_extract:
  Number of participants = 354


# MERGE DATASETS

In [83]:
df_merge = pd.merge(df_probe_extract, df_question_extract, on='Participant')
df_merge

,Participant,avg_response_to_tut,avg_correctness_score,avg_question_rt
0,593890eac6aa16000101f037,0.428571,0.7,9.13734
1,596e1af7a09655000197d4bb,0.000000,0.8,7.94115
2,5af835d8e19f8c00019e6dc0,0.000000,0.8,15.83652
3,5b07b71c68eff50001d1c859,0.285714,0.6,7.32070
4,5bd7971b0aac450001f951aa,0.571429,0.8,28.15962
...,...,...,...,...
349,6295270587394aac7ce600db,0.142857,0.7,12.53357
350,63038702fea51556c37c36cf,0.571429,0.7,7.36651
351,63d17cf704d2d7053d56b962,0.857143,0.8,8.28522
352,63d3fa78d12b38b131ef6b76,1.000000,0.9,7.50861


In [84]:
df_merge = pd.merge(df_merge, df_overall_extract, on='Participant')
df_merge

,Participant,avg_response_to_tut,avg_correctness_score,avg_question_rt,TextRTMean
0,593890eac6aa16000101f037,0.428571,0.7,9.13734,8.150127
1,596e1af7a09655000197d4bb,0.000000,0.8,7.94115,8.258325
2,5af835d8e19f8c00019e6dc0,0.000000,0.8,15.83652,15.013760
3,5b07b71c68eff50001d1c859,0.285714,0.6,7.32070,8.980250
4,5bd7971b0aac450001f951aa,0.571429,0.8,28.15962,15.064745
...,...,...,...,...,...
349,6295270587394aac7ce600db,0.142857,0.7,12.53357,10.930252
350,63038702fea51556c37c36cf,0.571429,0.7,7.36651,7.708367
351,63d17cf704d2d7053d56b962,0.857143,0.8,8.28522,8.862255
352,63d3fa78d12b38b131ef6b76,1.000000,0.9,7.50861,8.167373


In [88]:
# Rename column
df_merge.rename(columns={'TextRTMean': 'avg_text_rt'}, inplace=True)

df_merge

,Participant,avg_response_to_tut,avg_correctness_score,avg_question_rt,avg_text_rt
0,593890eac6aa16000101f037,0.428571,0.7,9.13734,8.150127
1,596e1af7a09655000197d4bb,0.000000,0.8,7.94115,8.258325
2,5af835d8e19f8c00019e6dc0,0.000000,0.8,15.83652,15.013760
3,5b07b71c68eff50001d1c859,0.285714,0.6,7.32070,8.980250
4,5bd7971b0aac450001f951aa,0.571429,0.8,28.15962,15.064745
...,...,...,...,...,...
349,6295270587394aac7ce600db,0.142857,0.7,12.53357,10.930252
350,63038702fea51556c37c36cf,0.571429,0.7,7.36651,7.708367
351,63d17cf704d2d7053d56b962,0.857143,0.8,8.28522,8.862255
352,63d3fa78d12b38b131ef6b76,1.000000,0.9,7.50861,8.167373


In [89]:
# df_merge.to_csv('comprehension_data.csv', index=False)

# COUNT NUMBER OF PARTICIPANTS AFTER MERGING

In [92]:
num_participants = df_merge['Participant'].nunique()
print ("Number of participants after merging datasets = ", num_participants)

Number of participants after merging datasets =  354
